In [1]:
%matplotlib inline

In [2]:
%run ./setup_notebook.ipynb  # Change path in setup_notebook if needed

In [3]:
import pickle
import run_functions as rf
from subsets_analysis import Subsets

In [4]:
# run_dict = {
#             'J048' : [27, 29, 30, 32], 
#             'RL048': [23, 24, 25, 28, 29]
#            }

# # just take the first session from the first mouse for now
# mouse = list(run_dict.keys())[0]
# run_number  = run_dict[mouse][0]

# pkl_path = '/home/jamesrowland/Documents/Code/Vape/run_pkls/'

# run_path = os.path.join(pkl_path, mouse, 'run{}.pkl'.format(run_number))

run_path = '/media/thijs/shoal/calcium_imaging_teaching/run27.pkl'  # Change path

In [5]:
## Load data:
with open (run_path, 'rb') as f:
    run = pickle.load(f)

In [6]:
# how many frames before and after the onset of lick trial to include in the kernal
pre_frames  = 5
post_frames = 9
fs = 5  # frame rate / plane
offset = 4

In [7]:
# the number of cells stimulated on each trial
trial_subsets = Subsets(run).trial_subsets

In [8]:
run = rf.filter_unhealthy_cells(run)
run = utils.lick_binner(run)

In [9]:
# run.__dict__.keys()

In [10]:
flu_array = utils.build_flu_array(run, run.trial_start, pre_frames, post_frames)

# remove frames with artifact during photostimulation
artifact_free = np.hstack((
                np.arange(pre_frames), 
                np.arange(pre_frames+offset, pre_frames+post_frames
                         )))

flu_array = flu_array[:, :, artifact_free]

In [11]:
### filter out trials
# remove trials where 150 cells were stimmed (longer and larger artifact)
non_150 = trial_subsets!=150
non_nan = ~np.isnan(np.mean(np.mean(flu_array,0),1))
trials_keep = non_150 & non_nan

In [12]:
## Define data class:
class Session():
    """Class that stores all relevant data of one recording session."""
    def __init__(self):
        self.flu_array = flu_array[:, trials_keep, :]  # cells X trials X frames
        self.trial_type = np.array(run.trial_type)[trials_keep]
        self.outcome = run.outcome[trials_keep]
        self.trial_stim = trial_subsets[trials_keep]  # number of stimulated cells per trial
        assert self.flu_array.shape[1] == len(self.trial_type) == len(self.outcome)
        self.n_cells, self.n_trials, self.n_frames = self.flu_array.shape

In [13]:
session = Session()  # initiate class

In [14]:
## Example of how to access class:
i_cell = 100
i_trial = 45
session.flu_array[i_cell, i_trial, :]

array([ 0.28437889,  0.41735151, -0.01537711,  0.11531141, -0.23521256,
       -0.15542518,  0.10384657,  0.07064252,  0.13862137, -0.48861152])

In [15]:
## Save session class as pickle file:
with open('session1.pkl', 'wb') as f:
    pickle.dump(session, f)
    
## Load session class from pickle file:
# with open('session1.pkl', 'rb') as f:
#     a = pickle.load(f)